In [41]:
import os
import base64
import io
import time
import math
import torch
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, ConcatDataset
from IPython.display import clear_output
from trainLoop import getPeriodicity, getCount, getPeriodicity, getStart, running_mean, MAE, f1score
from Model_inn import RepNet
from Dataset import getCombinedDataset
import matplotlib.pyplot as plt
from torchvision import transforms
import matplotlib
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display
from IPython.display import HTML
from IPython.display import Javascript
import cv2

In [2]:
import yt_dlp

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
# Load trained weights
model = RepNet(num_frames=64).to(device)
model.load_state_dict(torch.load('checkpoint/x3dbb10.pt', map_location=device), strict=False)
model.eval()

e:\downloaded_videos\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\downloaded_videos\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
e:\downloaded_videos\venv\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


RepNet(
  (resnetBase): ResNet50Bottom(
    (original_model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [33]:
testDataset = getCombinedDataset('countix/countix_test.csv',
                                   'downloaded_videos/',
                                   'test')

✅ Found: downloaded_videos/test\0CnQ-_e5Sl8.mp4
✅ Found: downloaded_videos/test\0CnQ-_e5Sl8.mp4
✅ Created dataset with 2 videos.


In [42]:
def test_model(model, test_set, batch_size):
    model.to(device)
    model.eval()
    
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=1, drop_last=False, shuffle=False)
    lossMAE = torch.nn.SmoothL1Loss()
    lossBCE = torch.nn.BCEWithLogitsLoss()
    
    total_mae = 0
    total_mae_count = 0
    total_fscore = 0
    num_batches = 0
    
    with torch.no_grad():
        pbar = tqdm(test_loader, total=len(test_loader), desc="Testing")
        
        for X, y in pbar:
            X = X.to(device).float()
            y1 = y.to(device).float()
            y2 = getPeriodicity(y1).to(device).float()
            
            y1pred, y2pred = model(X)
            loss1 = lossMAE(y1pred, y1)
            loss2 = lossBCE(y2pred, y2)
            
            count_pred = torch.sum((y2pred > 0) / (y1pred + 1e-1), 1)
            count = torch.sum((y2 > 0) / (y1 + 1e-1), 1)
            loss3 = lossMAE(count_pred, count)
            
            total_mae += loss1.item()
            total_mae_count += loss3.item()
            total_fscore += f1score(y2, (y2pred > 0).float())
            num_batches += 1
            
            pbar.set_postfix({
                'MAE_period': total_mae / num_batches,
                'MAE_count': total_mae_count / num_batches,
                'F1-score': total_fscore / num_batches
            })
            
            del X, y, y1, y2, y1pred, y2pred
    
    print(f"Test Results: MAE_period={total_mae / num_batches:.4f}, "
          f"MAE_count={total_mae_count / num_batches:.4f}, "
          f"F1-score={total_fscore / num_batches:.4f}")
    
    return total_mae / num_batches, total_mae_count / num_batches, total_fscore / num_batches

In [43]:
# Call the test function
test_mae, test_mae_count, test_f1 = test_model(model, testDataset, 2)
print(f"Final Test Metrics - MAE: {test_mae}, Count MAE: {test_mae_count}, F1-score: {test_f1}")

Testing: 100%|██████████| 1/1 [00:04<00:00,  4.60s/it, MAE_period=7.15, MAE_count=4.45, F1-score=0]

Test Results: MAE_period=7.1502, MAE_count=4.4455, F1-score=0.0000
Final Test Metrics - MAE: 7.1502251625061035, Count MAE: 4.445500373840332, F1-score: 0.0
